In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
from IPython.core.display import HTML; 
display(HTML("<style>.p-Widget.jp-OutputPrompt.jp-OutputArea-prompt:empty {padding: 0; border: 0;}</style>"));

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from collections import OrderedDict
from copy import deepcopy
from glob import glob

import torch
import torchvision.models as models
from torch.autograd import Variable
import torchvision.transforms as transforms

import os, sys
import json, pickle

sys.path.append('..')

In [4]:
sys.path.append('../model_options')
from feature_extraction import *
from model_options import *

In [5]:
torch.cuda.set_device(3)
torch.cuda.get_device_name(3)

'TITAN Xp'

### Model Metadata Build

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
model_options = get_model_options(model_type='imagenet', train_type='random')

In [10]:
output_file = 'model_metadata.json'
if os.path.exists(output_file):
    model_metadata = json.load(open(output_file,'r'))

if not os.path.exists(output_file):
    model_metadata = {}
    for model_option in tqdm(model_options):
    #for model_option in tqdm(list(model_options.keys())[:3]):
        model_name = model_options[model_option]['model_name']
        model = eval(model_options[model_option]['call'])
        model = model.eval()
        model = model.cuda()
        parameter_count = count_parameters(model)
        model_inputs = Variable(torch.randn(1,3,224,224)).cuda()
        feature_maps = get_all_feature_maps(model, model_inputs, flatten=False)
        model_layer_order = [model_layer for model_layer in feature_maps]
        feature_map_shapes = [feature_maps[model_layer].squeeze().shape for model_layer in feature_maps]
        feature_map_counts = [feature_maps[model_layer].reshape(1,-1).shape[1] for model_layer in feature_maps]
        total_feature_count = int(np.array(feature_map_counts).sum())
        model_metadata[model_name] = {'layer_count': len(feature_maps.keys()),
                                      'parameter_count': parameter_count,
                                      'total_feature_count': total_feature_count,
                                      'layer_metadata': {'model_layer_order': model_layer_order,
                                                         'feature_map_shapes': feature_map_shapes,
                                                         'feature_map_counts': feature_map_counts}}

    with open('model_metadata.json', 'w') as file:
        json.dump(model_metadata, file)

In [11]:
model_metadata_df = pd.read_json('model_metadata.json', orient='index')
model_metadata_df['model_name'] = model_metadata_df.index
model_metadata_df.drop(columns='layer_metadata', inplace=True)
model_metadata_df.to_csv('model_metadata.csv', index = None)

### Model Metadata By Model String

In [12]:
model_options = get_model_options(model_type='imagenet')
model_typology = pd.read_csv('model_typology.csv')
model_typology_dict = model_typology.set_index('model_name').to_dict('index')

In [14]:
model_metadata_bystring = {}
for model_option in tqdm(model_options):
#for model_option in tqdm(list(model_options.keys())[:3]):
    model_name = model_options[model_option]['model_name']
    train_type = model_options[model_option]['train_type']
    formal_model_name = model_typology_dict[model_name]['formal_model_name']
    model_metadata_bystring[model_option] = {'model_name': model_name, 'formal_model_name': formal_model_name,
                                             'train_type': train_type, **model_metadata[model_name]}

if 'save_by_string' in globals():
    with open('model_metadata_bystring.json', 'w') as file:
        json.dump(model_metadata_bystring, file)